# Model assessment

## Theory

Model assessment aims at quantifying the model generalization performance by estimating the test error. While there are analytical methods, such as the Bayesian information criterion (BIC) and the Akaike information criterion (AIC), numerical methods are widely used. 

In cases where enough data are available, data are divided into three parts: a training, a validation and a test set. Training set is used to fit the model, validation set is used to estimate the prediction error for model selection and parameter tuning, and test set is used for a final estimation of the generalization error. 

In cases where data are scarce, resampling methods are used.  Cross-validation (CV) direclty estimates the prediction error and boostrap estimates statistical accuracy. 

## Generalization error

Let $X$ be a vector on inputs, $Y$ the target variable, $f(X)$ the prediction model and $L(Y,f(X))$ the loss function. The *test error*, also named *generalization error*, is the expected error over the test set
$$
Err=E(Y,f(X)),
$$
where $X$ and $Y$ are randomly sample from their joint distribution. 
Training error is the average loss over the training set
$$
Err_{train}=\sum_{i=1}^N[L(y_i,f(x_i))]
$$

For regression problems, variables are quantitative and a typical loss is the squared error
$$
L(X)=(Y-f(X))^2,
$$ 
or the absolute error, $L(X)=|Y-f(X)|$. 

For qualitative variables, the output variable or response $G$ takes values $1, 2, \ldots, K$. Typical loss is the $0$-$1$ loss, $L(G,\hat{G}(X))=I(G\neq\hat{G}(X))$, or the log-likelihood, also named *cross-entropy loss* or *deviance*
$$
L(G,\hat{G}(X))=-2\sum_{k=1}^KI(G=k)\log p_k(X)=-2\log p_G(X). 
$$